In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from mocpy import MOC, WCS
import time
import numpy as np
from astropy import units as u
from astropy.table import Table
from astropy.coordinates import Angle, SkyCoord
import ipywidgets as widgets

t = Table.read('./2MASS-list-images.fits.gz', format='fits')
t['ra'][:10].quantity.shape

(10,)

In [2]:
import mocpy
mocpy.__version__

'0.8.0'

In [3]:
output = widgets.Output()

def update_plot(change):
    global t, output, time_slider
    global last_t1, last_t2
    global moc
    output.clear_output(wait=True)
    
    if change is None:
        new_values = time_slider.value
    else:
        new_values = change.new
    
    new_t1, new_t2 = new_values
    bigger_moc = True
    delta_t1 = 0
    delta_t2 = 0
    
    if last_t1 == new_t1:
        # The user touched the right slider
        if last_t2 < new_t2:
            # He moved the right slider to the right
            delta_t1 = last_t2
            delta_t2 = new_t2
        else:
            # He moved the right slider to the left
            delta_t1 = new_t2
            delta_t2 = last_t2
            bigger_moc = False
    else:
        # The user touched the left slider
        if last_t1 > new_t1:
            # He moved the left slider to the left
            delta_t1 = new_t1
            delta_t2 = last_t1
        else:
            # He moved the left slider to the right
            delta_t1 = last_t1
            delta_t2 = new_t1
            bigger_moc = False

    # Important remark: If the moc is smaller we have to recompute it from the beginning because cells 
    # can be observed in the new moc but would be removed in case they are observed in the delta moc too!
    if bigger_moc:
        selection = np.where((t['mjd'] >= delta_t1) & (t['mjd'] <= delta_t2))
        start = time.time()
        delta_moc = MOC.from_lonlat(t['ra'][selection].quantity, t['dec'][selection].quantity, max_norder=7)
        end = time.time()
        print("Finish moc creation of {0} items in {1} sec".format(selection[0].shape[0], end - start))
        moc = moc.union(delta_moc)
    else:
        selection = np.where((t['mjd'] >= new_t1) & (t['mjd'] <= new_t2))
        start = time.time()
        moc = MOC.from_lonlat(t['ra'][selection].quantity, t['dec'][selection].quantity, max_norder=7)
        end = time.time()
        print("Finish moc creation of {0} items in {1} sec".format(selection[0].shape[0], end - start))

    # Keep a trace of the time range for the next update
    last_t1, last_t2 = new_values
    with output:
        fig = plt.figure(111, figsize=(15, 10))
        
        with WCS(fig, 
                 fov=320 * u.deg,
                 center=SkyCoord(0, 0, unit='deg', frame='galactic'),
                 coordsys="galactic",
                 rotation=Angle(0, u.degree),
                 projection="AIT") as wcs:
            ax = fig.add_subplot(1, 1, 1, projection=wcs)
            # Call fill with a matplotlib axe and the `~astropy.wcs.WCS` wcs object.
            moc.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="green")
        
        plt.xlabel('GLON')
        plt.ylabel('GLAT')
        plt.title('Coverage of 2MASS images between MJD %d and %d' % (new_values[0], new_values[1]))
        plt.grid(color="black", linestyle="dotted")
        plt.show()

In [4]:
min_time = int(np.min(t['mjd']))-1
max_time = int(np.max(t['mjd']))+1

time_slider = widgets.IntRangeSlider(
    value=[min_time, min_time+50],
    min=min_time,
    max=max_time,
    step=1,
    description='MJD interval:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    layout={'width': '90%'}
)

In [5]:
last_t1 = 0
last_t2 = 0
moc = MOC()
time_slider.observe(update_plot, 'value')
update_plot(None)
widgets.VBox([time_slider, output])

Finish moc creation of 33120 items in 0.002582550048828125 sec


Finish moc creation of 2756880 items in 0.12046456336975098 sec
Finish moc creation of 22218 items in 0.0014426708221435547 sec
Finish moc creation of 4856910 items in 0.21308493614196777 sec
Finish moc creation of 0 items in 0.0002448558807373047 sec
Finish moc creation of 4879128 items in 0.21094012260437012 sec
Finish moc creation of 3937698 items in 0.15651464462280273 sec
Finish moc creation of 3176835 items in 0.13538336753845215 sec
Finish moc creation of 1917126 items in 0.08121633529663086 sec
Finish moc creation of 1184391 items in 0.04904794692993164 sec
Finish moc creation of 63453 items in 0.0030312538146972656 sec
Finish moc creation of 469641 items in 0.01766180992126465 sec
Finish moc creation of 66840 items in 0.003571033477783203 sec
Finish moc creation of 2692401 items in 0.11195516586303711 sec
Finish moc creation of 824757 items in 0.029279470443725586 sec
Finish moc creation of 4042020 items in 0.16283392906188965 sec
Finish moc creation of 4497972 items in 0.1793